In [ ]:
%load_ext autoreload
%autoreload 2
from openlm.tools import continuous_wavelet_transforms
import tifffile as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

### Load Image 1

In [ ]:
image_1 = tf.imread(r'C:\Users\User\Github\openCLEM\scratch\stitched.tif')
image_1 = image_1[300:1300, 400:1400]
plt.figure(dpi=200, figsize=(5,5))
plt.imshow(image_1, cmap='turbo')
plt.show()
print(image_1.shape)

### Load Image 2

In [ ]:
image = tf.imread(r"C:\Users\User\Downloads\eb_1_5.tif").astype(np.float32)
image = image[:, :, 0]
print(image.shape)
plt.figure(dpi=200, figsize=(5, 5))
plt.imshow(image, cmap='gray')
pixel_size = 800.e-6/image.shape[1]
print(image.shape[1])
print(pixel_size)

In [ ]:
# image = ndimage.median_filter(image, 3)
image = image[-1000:, 200:1200]
print(image.shape)
image = continuous_wavelet_transforms.normalise_image(image)
plt.figure(dpi=200, figsize=(5, 5))
plt.imshow(image, cmap='gray')

In [ ]:
tf.imwrite(r'C:\Users\User\Github\openCLEM\scratch\lm_1000.tif', image_1)

In [ ]:
# save
# pad image to 2500x2500
# image = np.pad(image, (((2500-image.shape[0])//2, (2500-image.shape[0])//2), ((2500-image.shape[1])//2, (2500-image.shape[1])//2)), mode='constant', constant_values=0)
print(image.shape)
plt.imshow(image)
plt.show()
tf.imwrite(r'C:\Users\User\Github\openCLEM\scratch\EB_1000.tif', image)

### Performing wavelet

In [ ]:
pixel_size = 1
feature_size = 24
buffer = 24
n_scales = 25

In [ ]:
coeffs, wav_norm, scales = continuous_wavelet_transforms.wavelet_transform(image, 'morlet', pixel_size, feature_size, buffer, n_scales)

In [ ]:
continuous_wavelet_transforms.plot_continuous_wavelet_transform(coeffs, scales)

In [ ]:
coeffs, wav_norm, scales = continuous_wavelet_transforms.wavelet_transform(image, 'dog', pixel_size, feature_size, buffer, n_scales)

In [ ]:
continuous_wavelet_transforms.plot_continuous_wavelet_transform(coeffs, scales)

In [ ]:
first_scale = np.abs(coeffs[:, :, 0])


In [ ]:
# make sure first_scale and image_1 are the same shape

shape_1 = image_1.shape
shape_2 = first_scale.shape

if shape_1 != shape_2:
    shape = (min(shape_1[0], shape_2[0]), min(shape_1[1], shape_2[1]))
    image_1 = image_1[:shape[0], :shape[1]]
    first_scale = first_scale[:shape[0], :shape[1]]

    


In [ ]:
print(image_1.shape)
print(first_scale.shape)

In [ ]:
import numpy as np
def crosscorrelation_v2(
    img1: np.ndarray, img2: np.ndarray, bandpass: np.ndarray = None
) -> np.ndarray:
    """
    Cross-correlate two images using Fourier convolution matching.

    Args:
        img1 (np.ndarray): The reference image.
        img2 (np.ndarray): The new image to be cross-correlated with the reference.
        bandpass (np.ndarray, optional): A bandpass mask to apply to both images before cross-correlation. Defaults to None.

    Returns:
        np.ndarray: The cross-correlation map between the two images.
    """
    if img1.shape != img2.shape:
        err = (
            f"Image 1 {img1.shape} and Image 2 {img2.shape} need to have the same shape"
        )
        raise ValueError(err)

    if bandpass is None:
        bandpass = np.ones_like(img1)

    n_pixels = img1.shape[0] * img1.shape[1]

    img1ft = np.fft.ifftshift(bandpass * np.fft.fftshift(np.fft.fft2(img1)))
    tmp = img1ft * np.conj(img1ft)
    img1ft = n_pixels * img1ft / np.sqrt(tmp.sum())

    img2ft = np.fft.ifftshift(bandpass * np.fft.fftshift(np.fft.fft2(img2)))
    img2ft[0, 0] = 0
    tmp = img2ft * np.conj(img2ft)

    img2ft = n_pixels * img2ft / np.sqrt(tmp.sum())

    # import matplotlib.pyplot as plt
    # fig, ax = plt.subplots(1, 2, figsize=(15, 15))
    # ax[0].imshow(np.fft.ifft2(img1ft).real)
    # ax[1].imshow(np.fft.ifft2(img2ft).real)
    # plt.show()

    # plt.title("Power Spectra")
    # plt.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(img1)))))
    # plt.show()

    xcorr = np.real(np.fft.fftshift(np.fft.ifft2(img1ft * np.conj(img2ft))))

    return xcorr


In [ ]:
result = crosscorrelation_v2(image_1, first_scale)


In [ ]:
plt.imshow(result, cmap='gray')

In [ ]:
plt.imshow(image_1)

In [ ]:
plt.imshow(first_scale)

### EB ATTEMPT 

In [ ]:
image = tf.imread(r"C:\Users\User\Documents\15-06-23 - Cells\eb_1_5.tif").astype(np.float32)
image = ndimage.median_filter(image, 3)
image = image[:, :, 0]
image = image[:1000, :1000]
image = continuous_wavelet_transforms.normalise_image(image)
plt.imshow(image, cmap='gray')

In [ ]:
coeffs, wav_norm, scales = continuous_wavelet_transforms.wavelet_transform(image, 'mexh', pixel_size, feature_size, buffer, n_scales)

In [ ]:
continuous_wavelet_transforms.plot_continuous_wavelet_transform(coeffs, scales)

### LM ATTEMPT

In [ ]:
plt.figure(dpi=200, figsize=(5,5))
plt.imshow(image_1, cmap='turbo')
plt.show()

In [ ]:
image_1_cont = continuous_wavelet_transforms.normalise_image(image_1)
image_1_cont = image_1_cont[200:2200, 200:2200]
print(image_1_cont.shape)
plt.imshow(image_1_cont)
# save image_1_cont
tf.imwrite(r'C:\Users\User\Github\openCLEM\scratch\image_1_cont.tif', image_1_cont)


In [ ]:
coeffs, wav_norm, scales = continuous_wavelet_transforms.wavelet_transform(image_1_cont, 'gaus', pixel_size, feature_size, buffer, n_scales)

In [ ]:
continuous_wavelet_transforms.plot_continuous_wavelet_transform(coeffs, scales)

In [ ]:
coeffs, wav_norm, scales = continuous_wavelet_transforms.wavelet_transform(image_1_cont, 'mexh', pixel_size, feature_size, buffer, n_scales)

In [ ]:
continuous_wavelet_transforms.plot_continuous_wavelet_transform(coeffs, scales)

In [ ]:
coeffs, wav_norm, scales = continuous_wavelet_transforms.wavelet_transform(image_1_cont, 'morlet', pixel_size, feature_size, buffer, n_scales)

In [ ]:
continuous_wavelet_transforms.plot_continuous_wavelet_transform(coeffs, scales)

In [ ]:
from fibsem.imaging import masks

In [ ]:
bandpass = masks.create_bandpass_mask(
    shape=[1024, 1024], lp=[220], hp=[52], sigma=3
)

In [ ]:
plt.imshow(bandpass)